<a href="https://colab.research.google.com/github/bc2026/DenoisingSIAIML/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, morphology, filters, measure
from skimage.morphology import remove_small_objects
from skimage.measure import regionprops
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.ndimage import label
from scipy.ndimage import center_of_mass, find_objects
from scipy.spatial.distance import pdist
from sklearn.model_selection import train_test_split

import cv2
from PIL import Image
import math
from scipy.signal import wiener
import tifffile as tiff
import glob
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
print("Imports successful.")
os.getcwd()

Mounted at /content/drive/
Imports successful.


'/content'

In [ ]:
xslx_files = glob.glob('/content/drive/MyDrive/Prof._Lee_Research/Process/Particle P1_C1_S1_11/result/*.xlsx')
tif_files = glob.glob('/content/drive/MyDrive/Prof._Lee_Research/cell/*.tif')
savingPath = '/content/drive/MyDrive/Prof._Lee_Research/Process/Particle P1_C1_S1_11/'
annoPath = savingPath + "annotated_frames/"

path_to_data = xslx_files[0]
path_to_tiff = tif_files[0]

dfs = pd.read_excel(path_to_data, sheet_name=None)
df =  dfs['Sheet1']

df_X = df['centroidGlobalColX']
df_Y = df['centroidGlobalRowY']

# Predefined radius for bounding box
r = 100
ret, images = cv2.imreadmulti(path_to_tiff, [], cv2.IMREAD_ANYCOLOR)

In [ ]:
import cv2
import numpy as np
import os

img_paths = []
label_paths = []

# Define the save path
os.makedirs(annoPath, exist_ok=True)  # Ensure the directory exists

if len(df_X) == len(df_Y):
  for i in range(len(df_X)):
    # Select the first image
    image = images[i]  # Assuming this is a PIL image or NumPy array
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)  # Convert to OpenCV format

    # Coordinates and parameters
    X = df_X[i]  # X-coordinate
    Y = df_Y[i]  # Y-coordinate
    r = 10  # Radius for rectangle size

    start_point = (int(X + r), int(Y + r))
    end_point = (int(X - r), int(Y - r))

    # Draw the rectangle
    color = (255, 0, 0)  # Blue in BGR
    thickness = 2  # Rectangle thickness
    annotated_image = cv2.rectangle(image, start_point, end_point, color, thickness)


    os.makedirs(f'{annoPath}png', exist_ok=True)  # Ensure the directory exists
    os.makedirs(f'{annoPath}txt', exist_ok=True)  # Ensure the directory exists


    # Save the annotated image
    annotated_image_path = os.path.join(f'{annoPath}png', f"annotated_frame_{i+1}.png")
    cv2.imwrite(annotated_image_path, annotated_image)


    img_height, img_width = image.shape[:2]

    x_center_norm = X / img_width
    y_center_norm = Y / img_height
    width_norm = (2*r)  / img_width
    height_norm = (2*r) / img_height



    with open(f'{annoPath}txt/annotated_frame_{i+1}.txt', "w") as f:
      f.write(f"0 {x_center_norm} {y_center_norm} {width_norm} {height_norm}\n")

    img_paths.append(annotated_image_path)
    label_paths.append(f'{annoPath}txt/annotated_frame_{i+1}.txt')

else: print("X and Y coords do not match")
